In [1]:
import seaborn as sns
import pylab
import pandas as pd
import numpy as np

# Reading Data

In [11]:
data = pd.read_csv('./wandb_export_2021-05-28T11 02 16.214-04 00.csv')
display(data)


,Name,State,End Time,Final/Average Final Performance,Final/Runtime (seconds),setting,method,dataset
0,baseline-multi_task-mnist_5t,finished,2021-05-14T08:16:16.000Z,0.999403,2787.591778,multi_task,baseline,mnist
1,baseline-multi_task-mnist_5t,finished,2021-05-14T11:18:03.000Z,0.999310,4162.125227,multi_task,baseline,mnist
2,baseline-multi_task-mnist_5t,finished,2021-05-14T12:41:01.000Z,0.999214,5082.302093,multi_task,baseline,mnist
3,baseline-multi_task-mnist_5t,finished,2021-05-13T14:51:37.000Z,0.999209,6023.456713,multi_task,baseline,mnist
4,baseline-multi_task-mnist_5t,finished,2021-05-13T16:47:40.000Z,0.999000,4634.909156,multi_task,baseline,mnist
...,...,...,...,...,...,...,...,...
2527,gdumb-multi_task-cifar100_10t,finished,2021-05-16T03:48:25.000Z,0.008263,15.330428,multi_task,gdumb,cifar100
2528,agem-task_incremental-cifar100_10t,finished,2021-05-14T09:47:35.000Z,0.007812,65.117600,task_incremental,agem,cifar100
2529,baseline-task_incremental-cifar100_10t,finished,2021-05-12T11:58:24.000Z,0.006086,616.431743,task_incremental,baseline,cifar100
2530,gem-iid-cifar100,finished,2021-05-14T06:23:49.000Z,0.006010,274.640365,iid,gem,cifar100


# Rename duplicated settings
1. Renames duplicate settings
2. Removes PNN + Class_incremental

In [12]:
settings = data['setting'].values
for i, setting in enumerate(settings):
    if setting == 'incremental_sl':
        settings[i] = "class_incremental"
    elif setting == 'task_incremental_sl':
        settings[i] = "task_incremental"
    elif setting == "multi_task_sl":
        settings[i] = "multi_task"
    elif setting == "traditional_sl":
        settings[i] = "iid"
data['setting'] = settings
data[(data['method'] == 'pnn') & (data['setting'] == 'class_incremental')] = np.nan
display(data)

,Name,State,End Time,Final/Average Final Performance,Final/Runtime (seconds),setting,method,dataset
0,baseline-multi_task-mnist_5t,finished,2021-05-14T08:16:16.000Z,0.999403,2787.591778,multi_task,baseline,mnist
1,baseline-multi_task-mnist_5t,finished,2021-05-14T11:18:03.000Z,0.999310,4162.125227,multi_task,baseline,mnist
2,baseline-multi_task-mnist_5t,finished,2021-05-14T12:41:01.000Z,0.999214,5082.302093,multi_task,baseline,mnist
3,baseline-multi_task-mnist_5t,finished,2021-05-13T14:51:37.000Z,0.999209,6023.456713,multi_task,baseline,mnist
4,baseline-multi_task-mnist_5t,finished,2021-05-13T16:47:40.000Z,0.999000,4634.909156,multi_task,baseline,mnist
...,...,...,...,...,...,...,...,...
2527,gdumb-multi_task-cifar100_10t,finished,2021-05-16T03:48:25.000Z,0.008263,15.330428,multi_task,gdumb,cifar100
2528,agem-task_incremental-cifar100_10t,finished,2021-05-14T09:47:35.000Z,0.007812,65.117600,task_incremental,agem,cifar100
2529,baseline-task_incremental-cifar100_10t,finished,2021-05-12T11:58:24.000Z,0.006086,616.431743,task_incremental,baseline,cifar100
2530,gem-iid-cifar100,finished,2021-05-14T06:23:49.000Z,0.006010,274.640365,iid,gem,cifar100


# Group data and prepare for latex

## Helper Functions

In [13]:
def join_mean_std(dataframe, columns, 
                  formatter=lambda x: "%04.1f\\tiny{$\\pm$%03.01f}" % (x[0] * 100, x[1] * 100),
                  axis=1):
    """ Joins two columns in pandas and formats them as a string of mean +- std
            dataframe: pandas DataFrame
            columns: the two columns to merge
            formatter: function to apply to the columns rowwise.
            axis: pandas axis where to execute the cuntion
    """
    return dataframe[columns].apply(formatter, axis=axis)

In [29]:
# How many runs to consider for doing the mean
TOPK=5

# Group by setting and dataset, and method extracts the top5 highest values and returns the mean and std
table = data.groupby(['setting', 'dataset', 'method']).agg(mean_score=('Final/Average Final Performance', lambda x: x.nlargest(TOPK).mean()),
                                                       std_score=('Final/Average Final Performance', lambda x: np.nanstd(x.nlargest(TOPK))))

# Remove 'class_incremental' and 'task_incremental'. The values will be re-added later as 'iid'
# into iid
table = table.loc[['class_incremental', 'task_incremental']]

# Join mean and std in a +- string
table['std_score'] = join_mean_std(table, ['mean_score', 'std_score'])
table['mean_score'] = table['std_score']
table.drop(columns='std_score', inplace=True)

# Undo grouping to obtain the final flat table
table = table.reset_index().pivot(index=["setting", "dataset"], columns='method', values='mean_score')

# ADD BACK TASK_INCREMENTAL AND CLASS_INCREMENTAL as IID

# Groups by setting and dataset, extracts the top5 highest values and returns the mean and std
all_merged_table = data.groupby(['setting', 'dataset']).agg(mean_score=('Final/Average Final Performance', lambda x: x.nlargest(TOPK).mean()),
                                            std_score=('Final/Average Final Performance', lambda x: np.nanstd(x.nlargest(TOPK))))

# Join mean and std in a +- string
all_merged_table['std_score'] = join_mean_std(all_merged_table, ['mean_score', 'std_score'])
all_merged_table['mean_score'] = all_merged_table['std_score']
all_merged_table.drop(columns='std_score', inplace=True)

# From 'all_merged_table' it picks 'iid' puts it as 'class_incremental' while 'multi_task' is put as 'task_incremental'
for i, v in table.iterrows():
    if i[0] == 'class_incremental':
        table.loc[i, 'iid'] = all_merged_table.loc[('iid', i[1])].values[0]
    elif i[0] == 'task_incremental':
        table.loc[i, 'iid'] = all_merged_table.loc[('multi_task', i[1])].values[0]


# Final flatten 
flat_table = table.reset_index().replace(np.nan, 'N/A', regex=True)
flat_table.columns = flat_table.columns.rename('')
        
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(flat_table)

,setting,dataset,agem,baseline,cwr_star,ewc,gdumb,gem,lwf,pnn,random_baseline,replay,synaptic_intelligence,iid
0,class_incremental,cifar10,18.5\tiny{$\pm$0.2},25.7\tiny{$\pm$0.5},11.0\tiny{$\pm$1.7},09.9\tiny{$\pm$0.0},51.0\tiny{$\pm$1.9},56.3\tiny{$\pm$1.6},16.1\tiny{$\pm$1.7},N/A,09.6\tiny{$\pm$0.2},46.5\tiny{$\pm$3.3},16.9\tiny{$\pm$1.3},73.6\tiny{$\pm$0.7}
1,class_incremental,cifar100,06.1\tiny{$\pm$0.4},10.2\tiny{$\pm$0.9},01.0\tiny{$\pm$0.0},01.0\tiny{$\pm$0.0},19.8\tiny{$\pm$1.4},22.6\tiny{$\pm$4.0},02.6\tiny{$\pm$0.4},N/A,01.0\tiny{$\pm$0.1},01.8\tiny{$\pm$0.0},01.6\tiny{$\pm$1.1},38.6\tiny{$\pm$0.9}
2,class_incremental,mnist,21.2\tiny{$\pm$2.6},56.3\tiny{$\pm$3.4},14.5\tiny{$\pm$4.7},09.8\tiny{$\pm$0.0},98.0\tiny{$\pm$0.2},98.9\tiny{$\pm$0.2},17.6\tiny{$\pm$2.6},N/A,09.9\tiny{$\pm$0.3},97.5\tiny{$\pm$0.7},12.8\tiny{$\pm$4.0},99.3\tiny{$\pm$0.0}
3,class_incremental,synbols,02.3\tiny{$\pm$0.0},15.7\tiny{$\pm$0.7},02.2\tiny{$\pm$0.0},02.1\tiny{$\pm$0.0},76.5\tiny{$\pm$5.0},65.3\tiny{$\pm$1.6},02.3\tiny{$\pm$0.0},N/A,02.1\tiny{$\pm$0.0},02.3\tiny{$\pm$0.1},02.0\tiny{$\pm$0.1},75.5\tiny{$\pm$3.0}
4,task_incremental,cifar10,17.3\tiny{$\pm$0.8},73.8\tiny{$\pm$1.6},11.3\tiny{$\pm$2.5},N/A,40.0\tiny{$\pm$0.6},54.4\tiny{$\pm$5.8},12.4\tiny{$\pm$2.6},62.3\tiny{$\pm$4.8},10.0\tiny{$\pm$0.2},44.4\tiny{$\pm$18.3},09.8\tiny{$\pm$0.3},91.1\tiny{$\pm$0.4}
5,task_incremental,cifar100,06.2\tiny{$\pm$0.5},42.1\tiny{$\pm$3.0},01.6\tiny{$\pm$1.0},N/A,18.4\tiny{$\pm$1.0},10.3\tiny{$\pm$6.3},02.0\tiny{$\pm$0.0},14.1\tiny{$\pm$4.5},01.0\tiny{$\pm$0.1},N/A,02.8\tiny{$\pm$2.5},65.1\tiny{$\pm$1.3}
6,task_incremental,mnist,19.9\tiny{$\pm$0.4},98.3\tiny{$\pm$0.7},13.7\tiny{$\pm$3.9},N/A,97.2\tiny{$\pm$0.2},98.3\tiny{$\pm$0.1},10.1\tiny{$\pm$0.0},76.9\tiny{$\pm$8.9},10.1\tiny{$\pm$0.2},98.6\tiny{$\pm$0.1},13.2\tiny{$\pm$4.4},99.9\tiny{$\pm$0.0}
7,task_incremental,synbols,06.5\tiny{$\pm$2.1},58.0\tiny{$\pm$5.4},02.2\tiny{$\pm$0.1},N/A,76.6\tiny{$\pm$0.3},12.4\tiny{$\pm$14.7},01.8\tiny{$\pm$0.1},24.5\tiny{$\pm$0.9},02.1\tiny{$\pm$0.0},46.9\tiny{$\pm$3.3},02.0\tiny{$\pm$0.0},88.7\tiny{$\pm$0.6}


In [16]:
def auto_get_columns_values(dataframe, threshold=0.5):
    """ Helper function. Returns hyperparameters and values from a dataframe. 
        hyperparameters are those values that change less often than the threshold
        args:
            dataframe: pandas DataFrame
            threshold: float. Threshold to identify an hyperparam
        return: list, list. List with hyperparameters and list with values
    """
    columns = []
    values = []
    total_len = dataframe.shape[0]
    for column in dataframe.columns:
        if np.unique(dataframe[column].values).shape[0] < threshold * total_len:
            columns.append(column)
        else:
            values.append(column)
    return columns, values

# Obtain hparams automatically
columns, values = auto_get_columns_values(flat_table, 0.6)
print("Hyperparameters:")
print(columns)
print("Results:")
print(values)


# Move 'iid' from values (results) to columns (hyperparameters).
if 'iid' in values:
    values.remove('iid')
    columns.append('iid')

# Massimo requested to remove EWC
values.remove('ewc')

# Move the random baseline to the end
values.remove('random_baseline')
values.append('random_baseline')
flat_table = flat_table[columns + values]
print("columns", columns, "values", values)


# More helper functions

def bold_best_func(dataframe, values, parse_number=lambda x: float(x.split("\\")[0])):
    """ Helper function to bold best results, it needs a parser because numbers are in format mean +- std.
        this is hacky and it should be fixed
    """
    dataframe = dataframe.copy()
    for v in values:
        dataframe[v] = dataframe[v].str.replace("N/A", "-1")
    for i, row in dataframe.iterrows():
        v = row[values]
        nums = [parse_number(x) for x in v.values]
        dataframe.loc[i][values[np.argmax(nums)]] = "\\textbf{%s}" % dataframe.loc[i][values[np.argmax(nums)]]
    for v in values:
        dataframe[v] = dataframe[v].str.replace("-1","N/A")
    return dataframe

def replace_value_list(lst, previous, new):
    """ Looks for a value in a list and replaces it for a new one. Assumes unique lists.
    """
    idx = lst.index(previous)
    lst[idx] = new
    return lst


# PANDAS2LATEX
def pandas2latex(dataframe, columns, values, multirow=True, resizebox=1.0, cell_padding='6pt', 
                 rowcolors=0, color="lightgray", hline=None, bold_best=True, rename_header={}, 
                 rename_column={}, caption="", label=""):  
    """ Converts a pandas DataFrame into latex.
        args:
            dataframe: pandas DataFrame to conver
            columns: names of the hyperparameters (str)
            values: names of the metrics (str)
            multirow: whether to group rows with the same hyperparam into a single multirow (bool)
            resizebox: resizes the table by (100 * resizebox)% in latex (float)
            cell_padding: white space between cells (str)
            rowcolors: how often to gray/white table rows (int)
            color: xcolor color to gray out (str)
            hline: put an hline everytime the most outer hyperparameter changes. 
                If 2, it will change when the first or the second most outer change. (int)
            bold_best: bold best result
            rename_header: dictionary to rename column titles (dict)
            rename_column: dictionary to rename column values. Ex: {'lr': (0.1, 'default')}, 
                            to change all the 0.1 lr values to 'default' string (dict)
            
    """
    
    df_columns = list(dataframe.columns)
    for c in df_columns:
        if c in rename_column and c in columns:
            for before, after in rename_column[c]:
                dataframe[c] = dataframe[c].str.replace(before, after)
        if c in rename_header:
            df_columns = replace_value_list(df_columns, c, rename_header[c])
            if c in columns:
                columns = replace_value_list(columns, c, rename_header[c])
            else:
                values = replace_value_list(values, c, rename_header[c])
        
    dataframe.columns = df_columns
        
    current_values = {c: dataframe[c].values[0] for c in columns}
#     print(current_values)
    current_count = {c: 0 for c in columns}
    total = []
    color_code = ""
    dataframe = bold_best_func(dataframe, values)
    for i, (idx, v) in enumerate(dataframe.iterrows()):
        rows = []
        idxmax = 0
        for j, column in enumerate(dataframe.columns):
            if rowcolors > 0:
                color_code = "\\cellcolor{white}"
            if column in current_values:
                rows.append(color_code)
                if current_values[column] != v[column]:
                    if current_count[column] > 1:
                        total[i - 1][list(columns).index(column)] = "\\multirow{-%d}{*}{%s%s}" %(current_count[column], color_code,current_values[column])
                    else:
                        total[i - 1][list(columns).index(column)] = "%s" %(current_values[column])
                    current_values[column] = v[column]
                    current_count[column] = 0    
                current_count[column] += 1
            else:
                rows.append(v[column])
        total.append(rows)
        
    for i, column in enumerate(columns):
        if current_count[column] > 1:
            total[-1][i] = "\\multirow{-%d}{*}{%s}" %(current_count[column], current_values[column])
        else:
            total[-1][i] = "%s" %current_values[column]

    body = ""
    for j, line in enumerate(total):
        cline = ""
        
        if hline is not None and j > 0 and j < len(total) - 1:
            for i, l in enumerate(line):
                if "multirow" in l:
                    cline = "\\hline"
                    break
        body += " & ".join(line) + "\\\\" + cline + "\n"

    header = ""
    header += " & ".join(list(dataframe.columns)) + "\\\\"
    preamble = ''
    if rowcolors > 0:
        preamble += "\\rowcolors{%d}{}{%s}" %(rowcolors, color)
    if cell_padding != '6pt':
        preamble += "\\setlength\\tabcolsep{%s}" %cell_padding
    latex_table = """\\begin{table}
    \\centering
    %s
    \\centerline{\\resizebox{%f\\textwidth}{!}{\\begin{tabular}{%s}
    \\toprule
    %s
    \\midrule
    %s
    \\bottomrule
    \\end{tabular}}}
    \\caption{%s}
    \\label{%s}
\\end{table}""" %(preamble, resizebox, 'l' * len(columns) + "c" * len(values) + "", header, body, caption, label)
    print(latex_table.replace("_", "\\_"))

columns ['setting', 'dataset', 'iid'] values ['agem', 'baseline', 'cwr_star', 'gdumb', 'gem', 'lwf', 'pnn', 'replay', 'synaptic_intelligence', 'random_baseline']


In [176]:
rename_header = {'synaptic_intelligence': 'SI', 'random_baseline': 'random', 'baseline': 'BaseMethod',
                'iid':'IID', 'agem': 'aGEM', 'cwr_star': 'CWRStar', 'gem': 'GEM', 'lwf':'LWF', 'pnn':'PNN', 
                 'replay':'Replay', 'setting': 'Setting', "method": 'Method', 'dataset': 'Dataset'}
rename_column = {'setting': [('class_incremental', 'CI'), ('task_incremental', 'TI')]}
pandas2latex(flat_table, columns, values, resizebox=1.25, cell_padding='1.5pt', rowcolors=1, hline=1, rename_header=rename_header, rename_column=rename_column, label="tab:cls_study")

\begin{table}
    \centering
    \rowcolors{1}{}{lightgray}\setlength\tabcolsep{1.5pt}
    \centerline{\resizebox{1.250000\textwidth}{!}{\begin{tabular}{lllcccccccccc}
    \toprule
    Setting & Dataset & IID & aGEM & BaseMethod & CWRStar & gdumb & GEM & LWF & PNN & Replay & SI & random\\
    \midrule
    \cellcolor{white} & cifar10 & 73.6\tiny{$\pm$0.7} & 18.5\tiny{$\pm$0.2} & 25.7\tiny{$\pm$0.5} & 11.0\tiny{$\pm$1.7} & 51.0\tiny{$\pm$1.9} & \textbf{56.3\tiny{$\pm$1.6}} & 16.1\tiny{$\pm$1.7} & N/A & 46.5\tiny{$\pm$3.3} & 16.9\tiny{$\pm$1.3} & 09.6\tiny{$\pm$0.2}\\
\cellcolor{white} & cifar100 & 38.6\tiny{$\pm$0.9} & 06.1\tiny{$\pm$0.4} & 10.2\tiny{$\pm$0.9} & 01.0\tiny{$\pm$0.0} & 19.8\tiny{$\pm$1.4} & \textbf{22.6\tiny{$\pm$4.0}} & 02.6\tiny{$\pm$0.4} & N/A & 01.8\tiny{$\pm$0.0} & 01.6\tiny{$\pm$1.1} & 01.0\tiny{$\pm$0.1}\\
\cellcolor{white} & mnist & 99.3\tiny{$\pm$0.0} & 21.2\tiny{$\pm$2.6} & 56.3\tiny{$\pm$3.4} & 14.5\tiny{$\pm$4.7} & 98.0\tiny{$\pm$0.2} & \textbf{98.9\tiny{$\pm$0

In [40]:
display(flat_table)

,setting,dataset,iid,agem,baseline,cwr_star,ewc,gdumb,gem,lwf,random,replay,SI
0,CI,cifar10,73.6 $_{\pm 0.7}$,18.3 $_{\pm 0.2}$,25.7 $_{\pm 0.5}$,10.1 $_{\pm 0.0}$,09.9 $_{\pm 0.0}$,51.0 $_{\pm 1.9}$,54.7 $_{\pm 2.5}$,16.1 $_{\pm 1.7}$,09.6 $_{\pm 0.2}$,46.5 $_{\pm 3.3}$,16.9 $_{\pm 1.3}$
1,CI,cifar100,38.6 $_{\pm 0.9}$,05.8 $_{\pm 0.6}$,10.2 $_{\pm 0.9}$,01.0 $_{\pm 0.0}$,01.0 $_{\pm 0.0}$,16.3 $_{\pm 6.4}$,13.0 $_{\pm 3.1}$,02.6 $_{\pm 0.4}$,01.0 $_{\pm 0.1}$,N/A,01.6 $_{\pm 1.1}$
2,CI,mnist,99.3 $_{\pm 0.0}$,21.2 $_{\pm 2.6}$,56.3 $_{\pm 3.4}$,10.4 $_{\pm 0.0}$,09.8 $_{\pm 0.0}$,86.6 $_{\pm 19.7}$,96.4 $_{\pm 1.0}$,17.6 $_{\pm 2.6}$,09.9 $_{\pm 0.3}$,54.0 $_{\pm 44.1}$,12.8 $_{\pm 4.0}$
3,CI,synbols,75.5 $_{\pm 3.0}$,02.2 $_{\pm 0.1}$,15.7 $_{\pm 0.7}$,02.2 $_{\pm 0.1}$,02.1 $_{\pm 0.0}$,02.2 $_{\pm 0.0}$,36.6 $_{\pm 3.3}$,02.3 $_{\pm 0.0}$,02.1 $_{\pm 0.0}$,02.2 $_{\pm 0.0}$,02.0 $_{\pm 0.1}$
4,TI,cifar10,91.1 $_{\pm 0.4}$,16.1 $_{\pm 1.7}$,73.8 $_{\pm 1.6}$,10.1 $_{\pm 0.0}$,N/A,36.2 $_{\pm 7.0}$,19.6 $_{\pm 11.8}$,12.4 $_{\pm 2.6}$,10.0 $_{\pm 0.2}$,50.0 $_{\pm 0.0}$,09.8 $_{\pm 0.3}$
5,TI,cifar100,65.1 $_{\pm 1.3}$,00.9 $_{\pm 0.1}$,42.1 $_{\pm 3.0}$,01.6 $_{\pm 1.0}$,N/A,09.6 $_{\pm 4.5}$,N/A,02.0 $_{\pm 0.0}$,01.0 $_{\pm 0.1}$,N/A,02.8 $_{\pm 2.5}$
6,TI,mnist,99.9 $_{\pm 0.0}$,N/A,98.3 $_{\pm 0.7}$,13.3 $_{\pm 4.3}$,N/A,83.2 $_{\pm 10.2}$,72.6 $_{\pm 0.0}$,10.1 $_{\pm 0.0}$,10.1 $_{\pm 0.2}$,N/A,13.2 $_{\pm 4.4}$
7,TI,synbols,88.7 $_{\pm 0.6}$,02.3 $_{\pm 0.0}$,58.0 $_{\pm 5.4}$,02.0 $_{\pm 0.0}$,N/A,02.2 $_{\pm 0.1}$,02.1 $_{\pm 0.0}$,01.8 $_{\pm 0.1}$,02.1 $_{\pm 0.0}$,N/A,02.0 $_{\pm 0.0}$


In [63]:
a= [0,1,2]
a.insert(1,10)
a

[0, 10, 1, 2]